In [1]:
# imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
import time

from statistics import mean, stdev
from pytrends.request import TrendReq
from urllib3.exceptions import MaxRetryError

from statsmodels.tsa.stattools import adfuller, kpss
import statsmodels.api as sm

from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

import warnings

# our modules
from plotSeries import *
from checkStationarity import *
from GoogleTrendsData import *
from makeSeriesStationary import *
from predictorsSelection import *
from dataPreProcessing import *
from tsModels import *
from rollingPredictionfuns import *
from fittedAndPredictedValuefuns import *
from bootstrapfuns import *

# 1. Read data (StatCan)

In [2]:
# Reading file
data = pd.read_csv('../../data/retailEcommercesales/retailEcommerceSales.csv',sep=',')

# data filter
retailEcommercesales = data[~data["Sales"].str.contains('unadjusted')]
retailEcommercesales = retailEcommercesales.filter(['REF_DATE','VALUE'])

# rename columns
retailEcommercesales = retailEcommercesales.rename(columns = {'REF_DATE': 'Date', 'VALUE': 'Ecommerce_sales'})
retailEcommercesales_ts = retailEcommercesales.copy()

#Changing data type of date
retailEcommercesales=retailEcommercesales.set_index(['Date'])
retailEcommercesales.index = pd.to_datetime(retailEcommercesales.index)
retailEcommercesales_ts = retailEcommercesales_ts.set_index('Date')
retailEcommercesales_ts.index = pd.to_datetime(retailEcommercesales_ts.index)

# calculate and add growth rate to the dataframe
retailEcommercesales_ts['Growth_rate'] = retailEcommercesales_ts.pct_change()
retailEcommercesales_ts = retailEcommercesales_ts.dropna() # removing NA

# check sttaionarity of time series
check_stationarity(retailEcommercesales_ts[['Growth_rate']])

FileNotFoundError: [Errno 2] No such file or directory: '../../data/retailEcommercesales/retailEcommerceSales.csv'

# 2. Get Google Trends data

In [3]:
# Reading keywords
keywords_csv = pd.read_csv('../../data/keywords_data/EECOMMERCE.csv')

# extracting timeseries dataframe
for index, row in keywords_csv.iterrows():
    #calling Pytrends
    data, queries, topics=ecommerce_trend(keywords=[str(row['Keyword'])],category=str(row['SubcatNo']))
    
    if index==0:
        FinalData, queries, topics=ecommerce_trend(keywords=[str(row['Keyword'])],category=str(row['SubcatNo']))
        FinalData.rename(columns = {'':str(row['Keyword'])}, inplace = True)    #column name for frame
        FinalData=FinalData.drop(columns=['isPartial'])   #remove column
    else:
        data, queries, topics=ecommerce_trend(keywords=[str(row['Keyword'])],category=str(row['SubcatNo']))
        data.rename(columns = {'':str(row['Keyword'])}, inplace = True)
        data=data.drop(columns=['isPartial'])
        data = data[str(row['Keyword'])]
        FinalData = FinalData.join(data)

In [4]:
# write data to csv
FinalData.to_csv('EcommerceKeywordTimeSeries.csv')

In [5]:
#pre-processing time series
ecommerce_keyword_ts = pd.read_csv('EcommerceKeywordTimeSeries.csv')
ecommerce_keyword_ts.rename(columns = {'date':'Date'}, inplace = True)
ecommerce_keyword_ts = ecommerce_keyword_ts.set_index(['Date'])
ecommerce_keyword_ts.index = pd.to_datetime(ecommerce_keyword_ts.index)
ecommerce_keyword_ts = ecommerce_keyword_ts.dropna()

# 3. Make data stationary

In [6]:
#extracting and removing trend
normalized_ecommerce_keyword = normalize(ecommerce_keyword_ts)
detrend_ecommerce_keyword = (detrend(normalized_ecommerce_keyword))
check_stationarity(detrend_ecommerce_keyword)

Series DoorDash is trend stationary, trend needs to be removed
Series square online is not stationary, differencing can be used to make it stationary
All other series are stationary


In [7]:
# response and predictors structing
predictor = make_predictors_df(detrend_ecommerce_keyword)
response = make_predictors_df(retailEcommercesales_ts[['Growth_rate']])

# 4. Model fit

In [8]:
# Random Forest

rf_predictors, rf_response = get_lag1_data_ecommerce(retailEcommercesales_ts, detrend_ecommerce_keyword, response_var='Growth_rate')

#Splitting data in training and testing
train_rf, test_rf, extra_rf = ts_train_test_split(rf_response, rf_predictors, test_size=0.2)

#Fitting RF model
rfmodel = randomForest_modelfit(train_rf, response_var='Growth_rate', n_trees=600)
x_rf, y_rf = train_rf.loc[:, ~train_rf.columns.isin(['Growth_rate'])], train_rf[['Growth_rate']]

# fitted growth rate
fitted_growthRate_rf = rfmodel.predict(x_rf)
fitted_growthRate_rf = pd.DataFrame(fitted_growthRate_rf, columns={'Fitted GrowthRate'})
fitted_growthRate_rf.index = train_rf.index

#Random Forest Predictions
rf_pred_ecommercegrowth = rolling_prediction_randomForest(train_rf, test_rf, extra_rf,dependent_var='Growth_rate',
                                              n_trees=600)

#rf_pred_retailgrowth

# Fitted and predicted retail sales value
pred_error, fig, data = fitted_and_predicted_sales_ecommerce(rfmodel, 
                                                    rf_pred_ecommercegrowth, 
                                                    retailEcommercesales, train_rf, test_rf, extra_rf)
pred_error

Prediction error: 365396.9182999891


365396.9182999891

In [9]:
# Bootstrap for GDP

# create bootstrap samples
bs_samples = get_bootstrap_samples(train_rf, n_sample=100, rs=None)  # rs = RandomState(1234)

# get predicted growth rate uisng bootstrap samples
# predicted_growth_df = pd.DataFrame(columns=list(range(0, len(bs_samples))))
# for i, df in enumerate(bs_samples):
#     predicted_growth_temp = rolling_prediction_randomForest(df, test_rf, extra_rf, dependent_var='Growth_rate',n_trees=100)
#     predicted_growth_df.iloc[:, i] = predicted_growth_temp[0]


In [ ]:
bs_samples